In [2]:
import numpy as np
np.random.seed(13)
import pandas as pd
import predictonlyup
import matplotlib.pyplot as plt
import math
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras import optimizers
from __future__ import division
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

def findMaxAcc(valacc,valloss):
	prevloss=valloss[0]
	for i in range(1,len(valloss)):
		if(valloss[i]>prevloss):
			return valacc[i-1]
	#print('Problems with find MaxAccuracy')
	return valacc[len(valacc)-1]

ticker='ADBE'
price= pd.read_csv('/home/andrea/Desktop/NLFF/DataSetIndexesLabeled/indexes'+ticker+'.csv')
if(ticker=='AAPL'):
    ticker='AAPL_tot'
sentimentVector =  pd.read_csv('SentimentNews/'+ticker+'.csv')

#alignment of data
maxdata=max(sentimentVector['initTime'])
mindata=min(sentimentVector['initTime'])
price=price[price['Unnamed: 0']>=mindata]
price=price[price['Unnamed: 0']<=maxdata]
#remove the time from data
sentimentVector=sentimentVector.drop(['Unnamed: 0', 'initTime' ], axis=1)
#2
#label because of the market and append values without datatime
#Important to set lenth of trend in this to see if news affect from today to today+tendwindowtime
#this is theorical (looks at difference betwwen price before news and price after news)because for make it real i have to change price.iloc[i-1]['open'] to price.iloc[i]['open']
#because the trader can buy only on price.iloc[i]['open'] that is today
trendwindowtime=[1,4,12,20,24,26,30,35,40,48,60]
for t in trendwindowtime:
    #1
#label because of the maket and append values without data
#simo theroy past trend
    x=[]
    y=[]
    print('============================================================')
    print('============================================================')
    print('Working on window:',t)
    
    for i in range(1,len(price)-t):
        y.append(np.sign(price.iloc[i+t]['close']-price.iloc[i-1]['open']))
        x.append(sentimentVector.iloc[i].values)
    x=np.array(x)
    y=np.array(y)
    y=(y+1)/2
    
    #Split betwwen train-validation and test
    train=0.8
    nt=math.ceil(len(x)*train)
    x_tv=[]
    y_tv=[]
    x_test=[]
    y_test=[]
    x_tv=x[:nt]
    y_tv=y[:nt]
    x_test=x[nt:]
    y_test=y[nt:]
    
    #Fairly sampling the test 50% 50%
    posindex=np.where( y_test == 1 )
    negindex=np.where( y_test == 0 )
    
    yindex=[]
    nindex=min(len(posindex[0]),len(negindex[0]))
    
    #for i in range(1,nindex):
    y_testnew=np.concatenate((y_test[posindex[0][0:nindex]],y_test[negindex[0][0:nindex]]))
    x_testnew=np.concatenate((x_test[posindex[0][0:nindex]],x_test[negindex[0][0:nindex]]))
    
    nfold=10
    kf = KFold(n_splits=nfold, random_state=13, shuffle=True)


    #model selection
    learningrate=[0.1,0.01,0.001,0.0001,0.00001]
    drop=[0,0.25,0.5]
    bestacc=0
    bestl=0
    bestmodel=None
    cvaccuracy=[]
    print('ModelSelection')
    for l in learningrate:
        
        #print(l)
        #crossvalidation
        opt=keras.optimizers.SGD(lr=l, momentum=0.0, decay=0.0, nesterov=False)
        model = Sequential()
        model.add(Dense(1, input_shape=(5,), activation='sigmoid'))
        
        model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
        cvaccuracy=[]
        for train_index, test_index in kf.split(x_tv):
            x_train, x_val = x_tv[train_index], x_tv[test_index]
            y_train, y_val = y_tv[train_index], y_tv[test_index]
            history=model.fit(x_train, y_train, epochs=20, verbose=0,batch_size=10,validation_data=(x_val, y_val))

            maxAcc=findMaxAcc(history.history['val_acc'],history.history['val_loss'])
            cvaccuracy.append(maxAcc)
        mcvaccuracy=sum(cvaccuracy)/len(cvaccuracy)
        
        if(mcvaccuracy>bestacc):
            bestacc=mcvaccuracy
            bestl=l
            bestmodel=model
    
    print('Best Accuracy on validation:',bestacc)
    print('BestLearning:',bestl)
    tot=[]
    print(bestmodel.evaluate(x=x_test, y=y_test, batch_size=10, verbose=0))
    print(bestmodel.evaluate(x=x_testnew, y=y_testnew, batch_size=10, verbose=0))
    print('LOOK HOW IT WORKS WITH TEST SET....')
    opt=keras.optimizers.SGD(lr=bestl, momentum=0.0, decay=0.0, nesterov=False)
    model = Sequential()
    model.add(Dense(1, input_shape=(5,), activation='sigmoid'))

    model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])
    model.fit(x_tv, y_tv, epochs=20, verbose=0,batch_size=10)
    evaluation=model.evaluate(x=x_test, y=y_test, batch_size=10, verbose=0)
    evaluation1=model.evaluate(x=x_testnew, y=y_testnew, batch_size=10, verbose=0)
    acc=evaluation[1]
    acc1=evaluation1[1]
    print('Accuracy on test set')
    print(acc)
    print(predictonlyup.alwaysUp(y_test))
    print('on new')
    print(acc1)
   

/home/andrea/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Working on window: 1
ModelSelection
Best Accuracy on validation: 0.539310276582577
BestLearning: 0.0001
[0.6937795065929869, 0.5014577354616744]
[0.696991960943481, 0.47839506310813223]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5102040860639022
0.5247813411078717
on new
0.4938271600339148
Working on window: 4
ModelSelection
Best Accuracy on validation: 0.5394160654090403
BestLearning: 1e-05
[0.7178633293213202, 0.5233918153925946]
[0.721149882035596, 0.5178571436997681]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.5467836264281245
0.5058479532163743
on new
0.5505952398691859
Working on window: 12
ModelSelection
Best Accuracy on validation: 0.5758426408488551
BestLearning: 0.1
[0.7172495046319151, 0.4692082148580607]
[0.7200123746711088, 0.46084337452628527]
LOOK HOW IT WORKS WITH TEST SET....
Accuracy on test set
0.51319648443429
0.5131964809384164
on new
0.5000000007181282
Working on window: 20
ModelSelection
Best Accuracy on validation: 0.61434641159048

In [ ]:
print(len(x_test))
print(len(x_testnew))